In [1]:
from library_dicom.dicom_processor.tools.flip import *
from library_dicom.dicom_processor.tools.create_mip import *
from radiomics.featureextractor import RadiomicsFeatureExtractor 
import os 
import SimpleITK as sitk 

In [ ]:
#get bigger diameter 

In [2]:
csv_flip_path = r'D:\FLIP_NIFTI\FLIP_metrics.csv'

with open(csv_flip_path, 'r') as csv_file :
    reader = csv.reader(csv_file, delimiter = ',') #liste pour chaque ligne 
    csv_data = []
    for row in reader :
        csv_data.append(row)
        
del csv_data[0] #enlever première ligne

In [3]:
csv_data[0]

['CR,014',
 '2017014',
 '1.2.250.1.38.2.1.102.140724093005.103468016',
 'cr_014_jul 25_2014.csv',
 'pet0',
 '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI/CR,014/pet0/1.2.250.1.38.2.1.102.140724093005.103468016_nifti_CT.nii',
 '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI/CR,014/pet0/1.2.250.1.38.2.1.102.140724093005.103468016_nifti_PT.nii',
 '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI/CR,014/pet0/1.2.250.1.38.2.1.102.140724093005.103468016_nifti_mask.nii',
 '20',
 '9.66']

In [4]:
#good path 
for row in csv_data : 
    ct_path = row[5]
    pt_path = row[6]
    mask_path = row[7]

    new_ct_path =  os.path.join( r'D:', '/'.join(ct_path.split('/')[-4:]))
    new_pt_path =  os.path.join( r'D:', '/'.join(pt_path.split('/')[-4:]))
    new_mask_path =  os.path.join( r'D:', '/'.join(mask_path.split('/')[-4:]))

    row[5] = new_ct_path
    row[6] = new_pt_path
    row[7] = new_mask_path 



In [5]:
csv_data[0]

['CR,014',
 '2017014',
 '1.2.250.1.38.2.1.102.140724093005.103468016',
 'cr_014_jul 25_2014.csv',
 'pet0',
 'D:FLIP_NIFTI/CR,014/pet0/1.2.250.1.38.2.1.102.140724093005.103468016_nifti_CT.nii',
 'D:FLIP_NIFTI/CR,014/pet0/1.2.250.1.38.2.1.102.140724093005.103468016_nifti_PT.nii',
 'D:FLIP_NIFTI/CR,014/pet0/1.2.250.1.38.2.1.102.140724093005.103468016_nifti_mask.nii',
 '20',
 '9.66']

In [7]:
diam_2d = []
diam_3d = []
for row in csv_data : 
    print(csv_data.index(row))
    diameter_2d, diameter_3d = extract_features(row[7], row[6], int(row[8]))
    diam_2d.append(diameter_2d)
    diam_3d.append(diameter_3d)


0
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
1
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
2
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
3
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
4
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
5
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
6
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
7
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
8
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
9
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
10
GLCM is symmetrical, therefore Sum Av

In [8]:
len(csv_data)

60

In [13]:
csv_data[0]

['CR,014',
 '2017014',
 '1.2.250.1.38.2.1.102.140724093005.103468016',
 'cr_014_jul 25_2014.csv',
 'pet0',
 'D:FLIP_NIFTI/CR,014/pet0/1.2.250.1.38.2.1.102.140724093005.103468016_nifti_CT.nii',
 'D:FLIP_NIFTI/CR,014/pet0/1.2.250.1.38.2.1.102.140724093005.103468016_nifti_PT.nii',
 'D:FLIP_NIFTI/CR,014/pet0/1.2.250.1.38.2.1.102.140724093005.103468016_nifti_mask.nii',
 '20',
 '9.66']

In [9]:
len(diam_2d)

60

In [10]:
len(diam_3d)

60

In [14]:
nifti_directory = r'D:\FLIP_NIFTI'

with open(os.path.join(nifti_directory, 'FLIP_metrics_v2.csv'), 'w') as csv_file : 
    csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(["PATIENT_NAME", "PATIENT_ID", "STUDY_UID", "CSV_NAME", "PET", "NIFTI_CT" ,"NIFTI_PET", "NIFTI_MASK", "ROI", "SUV_MAX", "DIAM_2D", "DIAM_3D"])
    for serie in csv_data: 
        index = csv_data.index(serie)
        csv_writer.writerow([serie[0], serie[1], serie[2], serie[3], serie[4], serie[5], serie[6], serie[7], serie[8], serie[9], diam_2d[index], diam_3d[index]])

In [8]:
mask_img = sitk.ReadImage(csv_data[0][7])
mask_array = sitk.GetArrayFromImage(mask_img).transpose()

pet_img = sitk.ReadImage(csv_data[0][6])
pet_array = sitk.GetArrayFromImage(pet_img).transpose()

In [9]:
mask_seuil = get_threshold_matrix_4D(mask_array, pet_array, 0.41 )

In [6]:
pet_img, mask_img = extract_features(csv_data[0][7], csv_data[0][6], 20)

In [7]:
mask_array = sitk.GetArrayFromImage(mask_img).transpose()
np.where(mask_array == 1)

(array([ 82,  83,  83, ..., 104, 104, 104], dtype=int64),
 array([113, 113, 113, ..., 127, 128, 128], dtype=int64),
 array([132, 124, 125, ...,  93,  87,  88], dtype=int64))

In [10]:
extractor = RadiomicsFeatureExtractor()
results = extractor.execute(pet_img, mask_img)

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


In [13]:
results

OrderedDict([('diagnostics_Versions_PyRadiomics', 'v3.0'),
             ('diagnostics_Versions_Numpy', '1.17.4'),
             ('diagnostics_Versions_SimpleITK', '1.2.3'),
             ('diagnostics_Versions_PyWavelet', '1.0.0'),
             ('diagnostics_Versions_Python', '3.7.4'),
             ('diagnostics_Configuration_Settings',
              {'minimumROIDimensions': 2,
               'minimumROISize': None,
               'normalize': False,
               'normalizeScale': 1,
               'removeOutliers': None,
               'resampledPixelSpacing': None,
               'interpolator': 'sitkBSpline',
               'preCrop': False,
               'padDistance': 5,
               'distances': [1],
               'force2D': False,
               'force2Ddimension': 0,
               'resegmentRange': None,
               'label': 1,
               'additionalInfo': True}),
             ('diagnostics_Configuration_EnabledImageTypes', {'Original': {}}),
             ('diagnost

In [16]:
results['original_shape_MajorAxisLength']

numpy.float64

In [25]:
arr = results['original_shape_Maximum3DDiameter']
float(arr)

144.3173135176973

In [ ]:
'original_shape_MajorAxisLength', 106.66151978502202)
('original_shape_Maximum3DDiameter', array(144.31731352))

In [13]:

csv_flip_path = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP/petIni.csv'

with open(csv_flip_path, 'r') as csv_file :
    reader = csv.reader(csv_file, delimiter = ';') #liste pour chaque ligne 
    csv_data = []
    for row in reader :
        csv_data.append(row)
        
del csv_data[0] #enlever première ligne


In [14]:
csv_data[0]

['CR,014',
 '2017014',
 '1.2.250.1.38.2.1.102.140724093005.103468016',
 '1',
 'cr_014_jul 25_2014.csv',
 '1',
 '5',
 '11.13',
 'Bone Marrow',
 '3.4',
 'Abdominal',
 '2.19',
 '3.05',
 '4.08',
 '2.61',
 '3.35',
 '1',
 '6.7',
 '3.46',
 '4',
 '3.54',
 '2']

In [4]:
for serie in csv_data:
    print(csv_data.index(serie))
    roi, suv_max = get_suv_max_value(serie[6], serie[5])
    serie.append(roi)
    serie.append(suv_max)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59


In [5]:
csv_data[0]

['CR,014',
 '2017014',
 '1.2.250.1.38.2.1.102.140724093005.103468016',
 'pet0',
 '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI/CR,014/pet0/1.2.250.1.38.2.1.102.140724093005.103468016_nifti_CT.nii',
 '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI/CR,014/pet0/1.2.250.1.38.2.1.102.140724093005.103468016_nifti_PT.nii',
 '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI/CR,014/pet0/1.2.250.1.38.2.1.102.140724093005.103468016_nifti_mask.nii',
 19,
 9.66]

In [15]:
csv_flip_path = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI/FLIP_SUV_MAX_BIGGER_ROI.csv'

with open(csv_flip_path, 'r') as csv_file :
    reader = csv.reader(csv_file, delimiter = ',') #liste pour chaque ligne 
    csv_data_2 = []
    for row in reader :
        csv_data_2.append(row)
        
del csv_data_2[0] #enlever première ligne


In [16]:
csv_data_2[1]

['CR,017',
 '2017017',
 '1.2.840.113619.2.354.3.3523896338.523.1419230918.308',
 'pet0',
 '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI/CR,017/pet0/1.2.840.113619.2.354.3.3523896338.523.1419230918.308_nifti_CT.nii',
 '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI/CR,017/pet0/1.2.840.113619.2.354.3.3523896338.523.1419230918.308_nifti_PT.nii',
 '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI/CR,017/pet0/1.2.840.113619.2.354.3.3523896338.523.1419230918.308_nifti_mask.nii',
 '3',
 '8.19']

In [17]:
for row in csv_data_2 : 
    for ligne in csv_data : 
        if row[1] in ligne : 
            row.append(ligne[4])

In [18]:
csv_data_2[0]

['CR,014',
 '2017014',
 '1.2.250.1.38.2.1.102.140724093005.103468016',
 'pet0',
 '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI/CR,014/pet0/1.2.250.1.38.2.1.102.140724093005.103468016_nifti_CT.nii',
 '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI/CR,014/pet0/1.2.250.1.38.2.1.102.140724093005.103468016_nifti_PT.nii',
 '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI/CR,014/pet0/1.2.250.1.38.2.1.102.140724093005.103468016_nifti_mask.nii',
 '20',
 '9.66',
 'cr_014_jul 25_2014.csv']

In [23]:
nifti_directory = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI'

with open(os.path.join(nifti_directory, 'FLIP_metrics.csv'), 'w') as csv_file : 
    csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(["PATIENT_NAME", "PATIENT_ID", "STUDY_UID", "CSV_NAME", "PET", "NIFTI_CT" ,"NIFTI_PET", "NIFTI_MASK", "ROI", "SUV_MAX"])
    for serie in csv_data_2: 
        csv_writer.writerow([serie[0], serie[1], serie[2], serie[-1], serie[3], serie[4], serie[5], serie[6], serie[7], serie[8]])